In [1]:
from sklearn.datasets import fetch_20newsgroups
from keras.layers import  Dropout, Dense
from keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import model_selection, naive_bayes, svm




In [18]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [19]:
path="/content/gdrive/MyDrive/Github/Detection_Violence/DL/"

In [20]:
path2="/content/gdrive/MyDrive/Github/LDA/"

In [21]:
def TFIDF(X_train, X_test,MAX_NB_WORDS=75000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with",str(np.array(X_train).shape[1]),"features")
    return (X_train,X_test)

# Violence classification based Deep Neural Network 

In [22]:
def Build_Model_DNN_Text(shape, nClasses, dropout=0.5):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 512 # number of nodes
    nLayers = 4 # number of  hidden layer

    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [6]:
import nltk 
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


# Collection Data - (Violence in Yemens Social Media)

In [23]:
Corpus = pd.read_csv(path+"Violence in Yemens Social Media.csv", sep=';')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
import nltk 
words = set(nltk.corpus.words.words())
Corpus = pd.read_csv(path2+"Violence in Yemens Social Media.csv", sep=';')
list_term=[]
for term in Corpus.Text:
    sent = " ".join(w for w in nltk.wordpunct_tokenize(term) if w.lower() in words or not w.isalpha())
    list_term.append(sent)
dic={'Text':list_term, 'Dominant_Topic':Corpus.Dominant_Topic}
Corpus=pd.DataFrame(dic)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Violence Classification  corpus into  with 5 Topic 

In [ ]:
Corpus=Corpus.iloc[:50000]


In [25]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(Corpus['Text'],Corpus['Dominant_Topic'],test_size=0.2)


In [26]:
X_train_tfidf,X_test_tfidf = TFIDF(X_train,X_test)


tf-idf with 6330 features


In [27]:
model_DNN = Build_Model_DNN_Text(X_train_tfidf.shape[1], 5)
model_DNN.fit(X_train_tfidf, y_train,
                              validation_data=(X_test_tfidf, y_test),
                              epochs=10,
                              batch_size=256,
                              verbose=2)

Epoch 1/10
241/241 - 7s - loss: 0.5882 - accuracy: 0.7664 - val_loss: 0.1601 - val_accuracy: 0.9574 - 7s/epoch - 30ms/step
Epoch 2/10
241/241 - 2s - loss: 0.1252 - accuracy: 0.9656 - val_loss: 0.1171 - val_accuracy: 0.9676 - 2s/epoch - 7ms/step
Epoch 3/10
241/241 - 2s - loss: 0.0792 - accuracy: 0.9791 - val_loss: 0.1105 - val_accuracy: 0.9713 - 2s/epoch - 7ms/step
Epoch 4/10
241/241 - 2s - loss: 0.0615 - accuracy: 0.9850 - val_loss: 0.1202 - val_accuracy: 0.9744 - 2s/epoch - 7ms/step
Epoch 5/10
241/241 - 2s - loss: 0.0520 - accuracy: 0.9872 - val_loss: 0.1144 - val_accuracy: 0.9749 - 2s/epoch - 7ms/step
Epoch 6/10
241/241 - 2s - loss: 0.0464 - accuracy: 0.9894 - val_loss: 0.1156 - val_accuracy: 0.9754 - 2s/epoch - 7ms/step
Epoch 7/10
241/241 - 2s - loss: 0.0411 - accuracy: 0.9907 - val_loss: 0.1160 - val_accuracy: 0.9746 - 2s/epoch - 7ms/step
Epoch 8/10
241/241 - 2s - loss: 0.0397 - accuracy: 0.9910 - val_loss: 0.1201 - val_accuracy: 0.9752 - 2s/epoch - 7ms/step
Epoch 9/10
241/241 - 2s

In [28]:
MAX_TEXT_LENGTH=46004
Prediction = np.argmax(model_DNN.predict(X_test_tfidf[:, :MAX_TEXT_LENGTH]),axis=1)



In [34]:
term_topic0=['violence', 'organization', 'government', 'practice', 'objective', 'social', 'religious', 'state', 'hundred', 'unlawful']
term_topic1=['terrorist', 'militia', 'people', 'yemeni', 'houthi', 'year', 'eman', 'girl', 'village', 'support']
term_topic3=['mine', 'international', 'home', 'plant', 'city', 'displace', 'terror', 'blow', 'threat', 'face']
term_topic4=['group', 'crime', 'life', 'take', 'racist', 'believe', 'call', 'control', 'send', 'carry']
term_topic5=['child', 'kill', 'recruit', 'political', 'houthis', 'civilian', 'war', 'woman', 'force', 'taiz']



In [35]:
Topic0="Religious organization violence"
Topic1="Houthi militias terrorize Yemeni people"
Topic3="international peace threats"
Topic4="Racist Tendencies"
Topic5="Recruitment and killing of children"

In [36]:
from sklearn.metrics import classification_report

target_names = [Topic0, Topic1, Topic3, Topic4, Topic5]
report=classification_report(y_test, Prediction, target_names=target_names, digits=2)
print(report)

                                         precision    recall  f1-score   support

        Religious organization violence       0.98      0.97      0.97      1318
Houthi militias terrorize Yemeni people       0.98      0.98      0.98      8465
            international peace threats       0.98      0.95      0.96      1117
                      Racist Tendencies       0.97      0.97      0.97      1983
    Recruitment and killing of children       0.96      0.96      0.96      2530

                               accuracy                           0.98     15413
                              macro avg       0.97      0.97      0.97     15413
                           weighted avg       0.98      0.98      0.98     15413

